In [ ]:
# import libaries

from PIL import Image
from tqdm.notebook import tqdm
import numpy as np
import random
import sys
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchinfo import summary
from torchvision import transforms
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import KFold

In [ ]:
print('Python version:', sys.version)
print('CUDA Available:', torch.cuda.is_available())

if torch.cuda.is_available():
    print('GPU Name:', torch.cuda.get_device_name())
    print('GPU Properties:\n', torch.cuda.get_device_properties('cuda'))
    device = "cuda"
    torch.cuda.set_per_process_memory_fraction(0.95, 0)
    torch.cuda.empty_cache()
else:
    print("Cuda is not available, please use cpu instead")
    device = "cpu"
!nvidia-smi

In [ ]:
seed = 41
# Define custom dataset
class FaceDataset(Dataset):
    def __init__(self, df = None, num_sample = None, transform = None, num_img_pool = 10):
        # set random seed for FaceDataset
        np.random.seed(seed)
        random.seed(seed)
        # create constructors
        self.unique_img_name = None
        self.data = dict()
        self.images = list()
        # label to indices
        self.label_to_indices = dict()
        self.labels = list()
        # read csv file
        self.df = df
        # set the transformation
        self.transform = transform
        # drop last n row from dataframe
        self.df = self.df.head(num_sample)
        #get the length of entire dataset
        self.len_ = len(self.df)
        # load imgs
        self.load_imgs(self.df, num_imgs = num_img_pool, max = num_sample)

    def __len__(self):
        return self.len_

    # get each pair of images -> 1: same identity, 0: different identity
    # if index is even -> same pair
    # if index is odd -> random identity
    def __getitem__(self, idx):
        anchor_img = self.images[idx]
        anchor_label = self.labels[idx]

        pos_idx = np.random.choice(np.arange(len(self.images))[self.labels == anchor_label])
        neg_idx = np.random.choice(np.arange(len(self.images))[self.labels != anchor_label])

        pos_img = self.images[pos_idx]
        neg_img = self.images[neg_idx]

        pos_label = self.labels[pos_idx]
        neg_label = self.labels[neg_idx]

        if self.transform is None:
            img_to_tensor = transforms.ToTensor()
            anchor_img = img_to_tensor(anchor_img)
            pos_img = img_to_tensor(pos_img)
            neg_img = img_to_tensor(neg_img)
        else:
            anchor_img = self.transform(anchor_img)
            pos_img = self.transform(pos_img)
            neg_img = self.transform(neg_img)

        return anchor_img, pos_img, neg_img

    # load imgs from pandas to memory and define the maximum number of images
    def load_imgs(self, df, num_imgs, max):
        # iterate thought each row
        for i, row in tqdm(df.iterrows(), total = max):
            # get identity of each row
            row_identity = row['identity']
            # append each identity to numberical value
            self.label_to_indices[int(row_identity)] = i
            count_img = 0
            # loop imgs in each identity
            for img_name in row['path']:
                if count_img > num_imgs:
                    break
                # concatenate the directoru and image name
                # path_to_image = self.dir+img_name
                path_to_image = img_name
                # open image and convert to RGB
                img = Image.open(path_to_image).convert('RGB')

                self.images.append(img)
                self.labels.append(i)
                count_img += 1
            # print('Added img '+ str(row_identity))
        self.labels = np.array(self.labels)

In [ ]:
ds_df = pd.read_csv('./digiface_csv_files/digi_all.csv')
ds_df = ds_df.groupby('identity')['path'].apply(list).reset_index()
ds_df

In [ ]:
from sklearn.model_selection import train_test_split

# splitting each dataset
train_df, eval_df = train_test_split(ds_df, test_size= 0.3, shuffle = True, random_state = seed)
val_df, test_df = train_test_split(eval_df, test_size = 0.4, shuffle = True, random_state = seed)

# print to check size of each dataset
print(f'Train Size: {len(train_df)}')
print(f'Val Size: {len(val_df)}')
print(f'Test Size: {len(test_df)}')

train_df.head(5)

In [ ]:
# define image size
img_size = 112

# define transformation for test set
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(img_size),
    transforms.RandomHorizontalFlip(p=0.6),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# define transformation for validation set
val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# define batch size
# train_batch_size = 64
# val_batch_size = 64
# print('------------Started Loading Train Set------------')
# # create dataloader for train set
# train_triplet_dataset = FaceDataset(df = train_df, num_sample = 6000, transform = train_transform)
train_triplet_dataloader = DataLoader(train_triplet_dataset, batch_size=train_batch_size, shuffle=True, pin_memory=True)
# print('Total Train Set: ', train_triplet_dataset.__len__())
# print('-----------Finished Loading Train Set------------')
#
# print('\n')
#
# print('------------Started Loading Validation Set------------')
# # create dataloader for validation set
# val_triplet_dataset = FaceDataset(df = val_df,num_sample = 3000, transform = val_transform)
# val_triplet_dataloader = DataLoader(val_triplet_dataset, batch_size=val_batch_size, shuffle=True, pin_memory=True)
# print('Total Train Set: ', val_triplet_dataset.__len__())
# print('-----------Finished Loading Validation Set------------')


In [ ]:
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, ReLU6, Sigmoid, Dropout2d\
    ,Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter

class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

class h_sigmoid(Module):
    def __init__(self, inplace=True):
        super(h_sigmoid, self).__init__()
        self.relu = ReLU6(inplace=inplace)

    def forward(self, x):
        return self.relu(x + 3) / 6


class h_swish(Module):
    def __init__(self, inplace=True):
        super(h_swish, self).__init__()
        self.sigmoid = h_sigmoid(inplace=inplace)

    def forward(self, x):
        return x * self.sigmoid(x)


class SELayer(Module):
    def __init__(self, channel, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = AdaptiveAvgPool2d(1)
        self.fc = Sequential(
            Linear(channel, channel // reduction),
            ReLU(inplace=True),
            Linear(channel // reduction, channel),
            h_sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

class PermutationBlock(Module):
    def __init__(self, groups):
        super(PermutationBlock, self).__init__()
        self.groups = groups

    def forward(self, input):
        n, c, h, w = input.size()
        G = self.groups
        output = input.view(n, G, c // G, h, w).permute(0, 2, 1, 3, 4).contiguous().view(n, c, h, w)
        return output

class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class Depth_Wise(Module):
    def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
    def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)

class LinearScheduler(nn.Module):
    def __init__(self, dropblock, start_value, stop_value, nr_steps):
        super(LinearScheduler, self).__init__()
        self.dropblock = dropblock
        self.i = 0
        self.drop_values = np.linspace(start=start_value, stop=stop_value, num=nr_steps)

    def forward(self, x):
        return self.dropblock(x)

    def step(self):
        if self.i < len(self.drop_values):
            self.dropblock.drop_prob = self.drop_values[self.i]

        self.i += 1

class DropBlock2D(nn.Module):
    r"""Randomly zeroes 2D spatial blocks of the input tensor.
    As described in the paper
    `DropBlock: A regularization method for convolutional networks`_ ,
    dropping whole blocks of feature map allows to remove semantic
    information as compared to regular dropout.
    Args:
        drop_prob (float): probability of an element to be dropped.
        block_size (int): size of the block to drop
    Shape:
        - Input: `(N, C, H, W)`
        - Output: `(N, C, H, W)`
    .. _DropBlock: A regularization method for convolutional networks:
       https://arxiv.org/abs/1810.12890
    """

    def __init__(self, drop_prob, block_size):
        super(DropBlock2D, self).__init__()

        self.drop_prob = drop_prob
        self.block_size = block_size

    def forward(self, x):
        # shape: (bsize, channels, height, width)

        assert x.dim() == 4, \
            "Expected input with 4 dimensions (bsize, channels, height, width)"

        if not self.training or self.drop_prob == 0.:
            return x
        else:
            # get gamma value
            gamma = self._compute_gamma(x)

            # sample mask
            mask = (torch.rand(x.shape[0], *x.shape[2:]) < gamma).float()

            # place mask on input device
            mask = mask.to(x.device)

            # compute block mask
            block_mask = self._compute_block_mask(mask)

            # apply block mask
            out = x * block_mask[:, None, :, :]

            # scale output
            out = out * block_mask.numel() / block_mask.sum()

            return out

    def _compute_block_mask(self, mask):
        block_mask = F.max_pool2d(input=mask[:, None, :, :],
                                  kernel_size=(self.block_size, self.block_size),
                                  stride=(1, 1),
                                  padding=self.block_size // 2)

        if self.block_size % 2 == 0:
            block_mask = block_mask[:, :, :-1, :-1]

        block_mask = 1 - block_mask.squeeze(1)

        return block_mask

    def _compute_gamma(self, x):
        return self.drop_prob / (self.block_size ** 2)


In [ ]:
class MobileFaceNet(Module):
    def __init__(self, embedding_size=512):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, embedding_size, bias=False)
        self.bn = BatchNorm1d(embedding_size)
        self.dropblock = DropBlock2D(block_size=3, drop_prob=0.3)
        self.dropout = Dropout2d(0.3)

    def forward_once(self,x):
        out = self.conv1(x)

        out = self.conv2_dw(out)

        out = self.conv_23(out)

        out = self.conv_3(out)

        out = self.conv_34(out)

        out = self.conv_4(out)

        # out = self.dropblock(out)

        out = self.conv_45(out)

        out = self.conv_5(out)

        out = self.conv_6_sep(out)

        out = self.dropblock(out)

        out = self.conv_6_dw(out)

        out = self.dropblock(out)

        out = self.conv_6_flatten(out)

        out = self.linear(out)

        out = self.bn(out)
        return l2_norm(out)

    def forward(self, anchor_img, positive_img, negative_img):
        anchor = self.forward_once(anchor_img)
        positive = self.forward_once(positive_img)
        negative = self.forward_once(negative_img)
        return anchor, positive, negative

class TripletLoss(nn.Module):
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        # calculate Euclidean's distance
        distance_positive = (anchor - positive).pow(2).sum(1)
        distance_negative = (anchor - negative).pow(2).sum(1)
        # distance_positive = F.pairwise_distance(anchor, positive).view((-1, 1))
        # distance_negative = F.pairwise_distance(anchor, negative).view((-1, 1))
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return torch.mean(losses)

def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))

    acc = float(tp + tn) / dist.size

    return acc

In [ ]:
triplet_model = MobileFaceNet()
print(summary(triplet_model, input_size=[(32,3,112,112),(32,3,112,112),(32,3,112,112)]))

In [ ]:
model_config = {
    'margin': 1.2,
    'lr': 1e-3,
    'patience': 5,
    'factor': 0.1,
    'min_lr': 1e-10,
    'threshold': 1e-3
}
# criterion = TripletLoss(margin = model_config['margin'])
# optimizer = optim.Adam(triplet_model.parameters(), lr=model_config['lr'])
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=model_config['patience'], factor=model_config['factor'], min_lr=model_config['min_lr'], threshold=model_config['threshold'], verbose=True)

In [ ]:
cross_val_dataset = FaceDataset(df = train_df, num_sample= 9000, transform=train_transform)

In [15]:
from sklearn.model_selection import KFold
from statistics import mean

img_size = 112
split = 5
batch_size = 64
cv_epoch = 120

k_fold = KFold(n_splits = 5, shuffle = True, random_state=seed)

history_cv = {
    'cv_train_loss': dict(),
    'cv_val_loss': dict(),
    'cv_train_acc': dict(),
    'cv_val_acc': dict()
}

# fold level
for fold, (train_idx, valid_idx) in tqdm(enumerate(k_fold.split(cross_val_dataset)),total = split):

    print(f'Fold: {fold}/{split}')

    # the number of epoch in each fold
    # Random get the sample from dataset
    train_subsampler = torch.utils.data.Subset(cross_val_dataset,train_idx)
    valid_subsampler = torch.utils.data.Subset(cross_val_dataset,valid_idx)

    # Load data into dataloader
    train_loader = DataLoader(cross_val_dataset, batch_size = batch_size,  shuffle = True,pin_memory=True)
    valid_loader = DataLoader(valid_subsampler, batch_size = batch_size, shuffle = True, pin_memory=True)

    # declare model
    triplet_cv_model = MobileFaceNet()
    triplet_cv_model.to(device)

    # loss function
    criterion = TripletLoss(margin = model_config['margin'])
    # optimizer function
    optimizer = optim.Adam(triplet_cv_model.parameters(), lr=model_config['lr'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=model_config['patience'], factor=model_config['factor'], min_lr=model_config['min_lr'], threshold=model_config['threshold'], verbose=True)

    epoch_train_loss = list()
    epoch_val_loss = list()

    epoch_train_acc = list()
    epoch_val_acc = list()
    print(f'train sample: {len(train_loader)}')
    print(f'val sample: {len(valid_loader)}')
    previous_lr = 0

    # epoch level
    for epoch in tqdm(range(cv_epoch)):

        cv_train_loss = 0
        cv_val_loss = 0

        triplet_cv_model.train()
        triplet_model.train()
        with torch.set_grad_enabled(True):
            # loop though each item in dataloader
            for anchor_img, pos_img, neg_img in train_loader:

                threshold_list = np.full(anchor_img.shape[0], model_config['margin'])
                is_same = np.full(anchor_img.shape[0], 1)

                # throw img to compute in the device
                anchor_img = anchor_img.to(device)
                pos_img = pos_img.to(device)
                neg_img = neg_img.to(device)
                # clear gradient to prevent gradient vanish
                optimizer.zero_grad()
                # train model
                output1, output2, output3 = triplet_model(anchor_img, pos_img, neg_img)

                # cal distance positive
                distance_positive = (output1 - output2).pow(2).sum(1)
                # compute loss from criterion
                loss = criterion(output1, output2, output3)
                # backward propagate
                loss.backward()
                # update parameters
                optimizer.step()
                # append train loss to epoch_train_loss
                cv_train_loss += loss.item() * anchor_img.size(0)

                epoch_train_acc = np.append(epoch_train_acc,[calculate_accuracy(threshold_list,distance_positive.cpu().data.numpy(),is_same)])

        cv_current_train_loss = cv_train_loss /  len(train_loader.sampler)

        epoch_train_loss.append(cv_current_train_loss)
        # get learning rate from model
        optim_lr = optimizer.param_groups[0]['lr']
        if (optim_lr < previous_lr) | (optim_lr > previous_lr):
            print('LEARNING RATE HAS CHANGED!')
        previous_lr = optim_lr
        print(f'Fold: {fold+1}, Epoch {epoch+1} / {cv_epoch}, learning rate: {optim_lr}, train loss: {round(cv_current_train_loss,5)},train acc: {round(epoch_train_acc.mean(),5)}')

        scheduler.step(cv_current_train_loss)

    history_cv['cv_train_loss'][fold] = epoch_train_loss
    history_cv['cv_val_loss'][fold] = epoch_val_loss

    history_cv['cv_train_acc'][fold] = epoch_train_acc.mean()
    history_cv['cv_val_acc'][fold] = epoch_val_acc.mean()
    print(f'------------------------Fold: {fold+1}/{split}--------------------------------')
    print(f'avg train loss: {round(mean(epoch_train_loss),5)}, avg val loss: {round(mean(epoch_val_loss),5)}, train acc: {round(epoch_train_acc.mean(),5)}, val acc: {round(epoch_val_acc.mean(),5)}')
    print('------------------------------------------------------------------------------')

  0%|          | 0/5 [00:00<?, ?it/s]

Fold: 0/5
train sample: 141
val sample: 29


  0%|          | 0/120 [00:00<?, ?it/s]

LEARNING RATE HAS CHANGED!
Fold: 1, Epoch 1 / 120, learning rate: 0.001, train loss: 1.05152,train acc: 0.08528
Fold: 1, Epoch 2 / 120, learning rate: 0.001, train loss: 1.05262,train acc: 0.08561
Fold: 1, Epoch 3 / 120, learning rate: 0.001, train loss: 1.05765,train acc: 0.08423
Fold: 1, Epoch 4 / 120, learning rate: 0.001, train loss: 1.05833,train acc: 0.08311
Fold: 1, Epoch 5 / 120, learning rate: 0.001, train loss: 1.05128,train acc: 0.08298
Fold: 1, Epoch 6 / 120, learning rate: 0.001, train loss: 1.05765,train acc: 0.08214
Fold: 1, Epoch 7 / 120, learning rate: 0.001, train loss: 1.05054,train acc: 0.08172
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
LEARNING RATE HAS CHANGED!
Fold: 1, Epoch 8 / 120, learning rate: 0.0001, train loss: 1.05211,train acc: 0.08204
Fold: 1, Epoch 9 / 120, learning rate: 0.0001, train loss: 1.04894,train acc: 0.08205
Fold: 1, Epoch 10 / 120, learning rate: 0.0001, train loss: 1.05559,train acc: 0.08202
Fold: 1, Epoch 11 / 120, learn

KeyboardInterrupt: 

In [ ]:
history_cv

In [ ]:
def train_nn(load_weight_pth = None, save_weight_pth = None, num_epochs = 200 , load_weight = False):
    load_weight_path  = './weights/triplet_mobilenetwork_100_epochs_.pth' if (load_weight_pth == None) else load_weight_pth
    save_weight_path = './weights/triplet_mobilenetwork_200_epochs_.pth' if (load_weight_pth == None) else save_weight_pth

    # throw model to compute in the device
    triplet_model.to(device=device)
    total_train_loss = list()
    total_val_loss = list()

    total_train_acc = list()
    total_val_acc = list()

    best_val_loss = 0
    best_train_loss = 0
    previous_lr = 0

    # load weight
    if bool(load_weight_path) & load_weight:
        triplet_model.load_state_dict(torch.load(load_weight_path))
        print('Congratulations, Weight has been loaded!')

    # iterate though each epoch
    for epoch in tqdm(range(num_epochs)):
        print(f'Epoch:{epoch+1}/{num_epochs}')
        # epoch_train_loss = list()
        epoch_train_acc = np.array([])
        # epoch_val_loss = list()
        epoch_val_acc = np.array([])

        train_loss = 0
        val_loss = 0

        # switch to train mode
        triplet_model.train()
        with torch.set_grad_enabled(True):
            # loop though each item in dataloader
            for anchor_img, pos_img, neg_img in tqdm(train_triplet_dataloader):

                threshold_list = np.full(anchor_img.shape[0], model_config['margin'])
                is_same = np.full(anchor_img.shape[0], 1)

                # throw img to compute in the device
                anchor_img = anchor_img.to(device)
                pos_img = pos_img.to(device)
                neg_img = neg_img.to(device)
                # clear gradient to prevent gradient vanish
                optimizer.zero_grad()
                # train model
                output1, output2, output3 = triplet_model(anchor_img, pos_img, neg_img)

                # cal distance positive
                distance_positive = (output1 - output2).pow(2).sum(1)
                # compute loss from criterion
                loss = criterion(output1, output2, output3)
                # backward propagate
                loss.backward()
                # update parameters
                optimizer.step()
                # append train loss to epoch_train_loss
                train_loss += loss.item() * anchor_img.size(0)

                epoch_train_acc = np.append(epoch_train_acc,[calculate_accuracy(threshold_list,distance_positive.cpu().data.numpy(),is_same)])



        # switch mode to eval
        triplet_model.eval()
        with torch.no_grad():
            for anchor_img, pos_img, neg_img in val_triplet_dataloader:

                threshold_list = np.full(anchor_img.shape[0], model_config['margin'])
                is_same = np.full(anchor_img.shape[0], 1)

                # throw img to compute in the device
                anchor_img = anchor_img.to(device)
                pos_img = pos_img.to(device)
                neg_img = neg_img.to(device)
                # clear gradient to prevent gradient vanish
                optimizer.zero_grad()
                # compute the out in eval mode
                output1, output2, output3 = triplet_model(anchor_img, pos_img, neg_img)
                # cal distance positive
                distance_positive = (output1 - output2).pow(2).sum(1)
                # compute loss in eval mode
                loss = criterion(output1, output2, output3)
                # calculate loss for each feature
                val_loss += loss.item() * anchor_img.size(0)

                # epoch_val_acc.append(calculate_accuracy(threshold_list,distance_positive.cpu().data.numpy(),is_same)[0])
                epoch_val_acc = np.append(epoch_val_acc,[calculate_accuracy(threshold_list,distance_positive.cpu().data.numpy(),is_same)])

        # calculate loss
        current_train_loss = train_loss / len(train_triplet_dataloader.sampler)
        total_train_loss.append(current_train_loss)

        # calculate loss for each batch
        current_val_loss = val_loss / len(val_triplet_dataloader.sampler)
        total_val_loss.append(current_val_loss)

        total_train_acc.append(epoch_train_acc.mean())
        total_val_acc.append(epoch_val_acc.mean())
        # get learning rate from model
        optim_lr = optimizer.param_groups[0]['lr']

        # append current validation loss to list
        if (best_val_loss == 0) | (best_train_loss == 0):
            best_val_loss = current_val_loss
            best_train_loss = current_train_loss

        # find the best val loss
        if best_val_loss >= current_val_loss:
            best_val_loss = current_val_loss

        # find the best train loss
        if best_train_loss >= current_train_loss:
            best_train_loss = current_train_loss

        print(f'train loss: {current_train_loss}')
        print(f'val loss: {current_val_loss}')
        print(f'Learning rate: {optim_lr:.8f}')
        print(f'acc train: {epoch_train_acc.mean()}')
        print(f'acc val: {epoch_val_acc.mean()}')
        if (optim_lr < previous_lr) | (optim_lr > previous_lr):
            print('LEARNING RATE HAS CHANGED!')
        print('-------------------------------------------------------------')

        # replace the previous lr with the current lr
        previous_lr = optim_lr
        scheduler.step(current_train_loss)

    print('Best Validation loss',best_val_loss)
    print('Best Train loss', best_train_loss)

    # save model weights
    torch.save(triplet_model.state_dict(), save_weight_path)
    return total_train_loss, total_val_loss, total_train_acc, total_val_acc

In [ ]:
# total_train_loss, total_val_loss, total_train_acc, total_val_acc = train_nn(num_epochs=120)

In [ ]:
plt.plot(np.arange(1,len(total_train_loss)+1), total_train_loss,label = 'Train Loss')
plt.plot(np.arange(1,len(total_val_loss)+1),total_val_loss, label= 'Validation loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(1,len(total_train_acc)+1), total_train_acc, label = 'Train Acc')
plt.plot(np.arange(1,len(total_val_acc)+1), total_val_acc, label = 'Validation Acc')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(1,len(total_train_acc)+1), total_train_acc, label = 'Train Acc')
plt.plot(np.arange(1,len(total_val_acc)+1), total_val_acc, label = 'Validation Acc')
plt.plot(np.arange(1,len(total_train_loss)+1), total_train_loss,label = 'Train Loss')
plt.plot(np.arange(1,len(total_val_loss)+1),total_val_loss, label= 'Validation loss')
plt.legend()
plt.show()

In [ ]:
# define batch size for testing
test_batch_size = 64
# create dataloader for validation set
test_triplet_dataset = FaceDataset(df = val_df, num_sample = 3000)
test_triplet_dataloader = DataLoader(test_triplet_dataset, batch_size=test_batch_size, shuffle=True, pin_memory=True)

In [ ]:
# define path
weights_path = './weights/triplet_mobilenetwork_200_epochs_.pth'

triplet_model.to(device)
# load model weights
triplet_model.load_state_dict(torch.load(weights_path))

overall_acc = list()

# set mode for testing
triplet_model.eval()
with torch.no_grad():
    for anchor_img, pos_img, neg_img in tqdm(test_triplet_dataloader):

        threshold_list = np.full(anchor_img.shape[0], model_config['margin'])
        is_same = np.full(anchor_img.shape[0], 1)

        pos_dist_list = list()
        model_output = list()

        anchor_img = anchor_img.to(device)
        pos_img = pos_img.to(device)
        neg_img = neg_img.to(device)
        # compute the out in eval mode
        anchor_img, pos_img, neg_img = triplet_model(anchor_img, pos_img, neg_img)
        # compute loss in eval mode
        loss = criterion(anchor_img, pos_img, neg_img)

        distance_positive = (anchor_img - pos_img).pow(2).sum(1)
        distance_negative = (anchor_img - neg_img).pow(2).sum(1)
        # print((distance_negative-distance_positive>0).sum()*1.0 / distance_negative.size()[0])
        # print(distance_positive.cpu().data.numpy())
        acc = calculate_accuracy(threshold_list,distance_positive.cpu().data.numpy(),is_same)
        overall_acc.append(acc)

print(f'Train Accuracy: {np.array(overall_acc).mean()}')
